In [4]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr

def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def calculate_stochastic_rsi(data, window=14):
    rsi = calculate_rsi(data, window)
    stoch_rsi = (rsi - rsi.rolling(window=window).min()) / (rsi.rolling(window=window).max() - rsi.rolling(window=window).min())
    return stoch_rsi

def get_stock_data(ticker, start_date, end_date):
    stock_data = fdr.DataReader(ticker, start_date, end_date)
    return stock_data

def main(ticker, start_date, end_date, rsi_window=14, stoch_rsi_window=14):
    data = get_stock_data(ticker, start_date, end_date)
    
    data['RSI'] = calculate_rsi(data, rsi_window)
    data['Stochastic RSI'] = calculate_stochastic_rsi(data, stoch_rsi_window)
    
    return data

# 예시 사용법
ticker = '005930'  # 삼성전자 종목 코드
start_date = '2020-01-01'
end_date = '2024-08-01'

stock_data = main(ticker, start_date, end_date)
print(stock_data[['RSI', 'Stochastic RSI']])
stock_data.to_csv("./test.csv")


                  RSI  Stochastic RSI
Date                                 
2020-01-02        NaN             NaN
2020-01-03        NaN             NaN
2020-01-06        NaN             NaN
2020-01-07        NaN             NaN
2020-01-08        NaN             NaN
...               ...             ...
2024-07-26  30.994152        0.012343
2024-07-29  30.588235        0.006208
2024-07-30  30.232558        0.000832
2024-07-31  40.703518        0.240853
2024-08-01  46.285714        0.368583

[1132 rows x 2 columns]
